# Parameter recovery with LBA model

An important test for any model is to make sure that parameters are identifiable in the context of the task we are simulating. Here, we'll define a task and simulate it using a simple linear ballistic accumulator (LBA) model. Then we'll use Bayesian sampling to estimate the posterior probability of each parameter.

## Define a task to run

For this example, we'll assume only one trial type. Each trial involves selecting between one of two choices. For each trial, we could specify which choice is correct. We would then have the model evaluate the probability of making each response. The other possibility is to simply assign, on each trial, one accumulator to the correct response and one to the incorrect response.

Simulating responses directly may be helpful if you want to account for participant bias to select one response or the other. Here, for simplicity, we'll ignore bias. Therefore, to keep things simple we'll have a "correct" accumulator and an "incorrect" accumulator.

Therefore, for each trial, we need to generate a simulated response time in seconds and a response (1: correct, 2: incorrect).

In [1]:
%matplotlib inline
import os
import sys
import numpy as np
module_path = os.path.abspath('..')
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
n_trial = 200
test_type = np.ones(n_trial)

In [5]:
import theano
import theano.tensor as tt

k = tt.iscalar('k')
v = tt.dvector('v')
v_c = tt.dscalar('v_c')
v_n = tt.dvector('v_n')
io = [1, 2, 3]

def inner_fct(prior_result, v):
    return prior_result * v

# Symbolic description of the result
result, updates = theano.scan(fn=inner_fct,
                              outputs_info=tt.ones_like(v_n),
                              sequences=[{'input': v, 'taps': io}],
                              non_sequences=[v_c],
                              n_steps=len(io))


TypeError: inner_fct() takes 2 positional arguments but 5 were given